In [28]:
import json
import os

In [30]:
pos_template = lambda adjective: f"I thought this movie was{adjective}, I loved it. The movie's genre of "
neg_template = lambda adjective: f"I thought this movie was{adjective}, I hated it. The movie's genre of "

pos_conditional_interventional_template = lambda adjective: f"I thought this movie was{adjective}, I loved it. The movie's genre of "
neg_conditional_interventional_template = lambda adjective: f"I thought this movie was{adjective}, I hated it. The movie's genre of "

positive_adjectives = [
    ' perfect', ' fantastic',' delightful',' cheerful',' marvelous',' good',' remarkable',' wonderful',
    ' fabulous',' outstanding',' awesome',' exceptional',' incredible',' extraordinary',
    ' amazing',' lovely',' brilliant',' charming',' terrific',' superb',' spectacular',' great',' splendid',
    ' beautiful',' joyful',' positive',#' excellent'
    ]

negative_adjectives = [
    ' dreadful',' bad',' dull',' depressing',' miserable',' tragic',' nasty',' inferior',' horrific',' terrible',
    ' ugly',' disgusting',' disastrous',' horrendous',' annoying',' boring',' offensive',' frustrating',' wretched',' dire',
    ' awful',' unpleasant',' horrible',' mediocre',' disappointing',' inadequate'
    ]

baseline_prompt = "Finish this review:\n"
conditional_prompt = lambda sentiment_polarity: f"Finish this review with {sentiment_polarity} sentiment:\n"
interventional_prompt = lambda sentiment_polarity: f"Rewrite this review with {sentiment_polarity} sentiment while maintaining the sentence structure, then finish the review:\n"

def get_prompts(adjectives, template, conditional_interventional_template, baseline_prompt, conditional_prompt, interventional_prompt, sentiment="positive"):
    prompts = []
    
    if sentiment == "positive":
        flipped_sentiment = "negative"
    else:
        flipped_sentiment = "positive"
        
    for adjective in adjectives:
        input_example_baseline = template(adjective)
        input_example_conditional_interventional = conditional_interventional_template(adjective)
        
        baseline_query = baseline_prompt + input_example_baseline
        conditional_query = conditional_prompt(flipped_sentiment) + input_example_conditional_interventional
        interventional_query = interventional_prompt(flipped_sentiment) + input_example_conditional_interventional

        prompts.append({
            "adjective": adjective,
            "sentiment": sentiment,
            "flipped_sentiment": flipped_sentiment,
            "baseline_query": baseline_query,
            "conditional_query": conditional_query,
            "interventional_query": interventional_query
        })
    
    return prompts


prompts_pos = get_prompts(positive_adjectives, pos_template, pos_conditional_interventional_template, baseline_prompt, conditional_prompt, interventional_prompt, sentiment="positive")
prompts_neg = get_prompts(negative_adjectives, neg_template, neg_conditional_interventional_template, baseline_prompt, conditional_prompt, interventional_prompt,sentiment="negative")
print(prompts_pos)
print(prompts_neg)

all_prompts = prompts_pos + prompts_neg
print(len(all_prompts))

json.dump(all_prompts, open("prompts.json", "w"), indent=2)



[{'adjective': ' perfect', 'sentiment': 'positive', 'flipped_sentiment': 'negative', 'baseline_query': "Finish this review:\nI thought this movie was perfect, I loved it. The movie's genre of ", 'conditional_query': "Finish this review with negative sentiment:\nI thought this movie was perfect, I loved it. The movie's genre of ", 'interventional_query': "Rewrite this review with negative sentiment while maintaining the sentence structure, then finish the review:\nI thought this movie was perfect, I loved it. The movie's genre of "}, {'adjective': ' fantastic', 'sentiment': 'positive', 'flipped_sentiment': 'negative', 'baseline_query': "Finish this review:\nI thought this movie was fantastic, I loved it. The movie's genre of ", 'conditional_query': "Finish this review with negative sentiment:\nI thought this movie was fantastic, I loved it. The movie's genre of ", 'interventional_query': "Rewrite this review with negative sentiment while maintaining the sentence structure, then finish t

In [2]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch

/net/scratch/garbacea/miniconda3/envs/editeval/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
device = 'cuda:0'
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
messages=[{"role": "user", "content": "Finish this review:\nI thought this movie was perfect, I loved it. The movie\'s genre of" }]
          #{"role": "user", "content": "Finish this review with positive sentiment:\nI thought this movie was perfect, I loved it. The movie's genre of "}, ]
          # {"role": "user", "content": "Rewrite this review with negative sentiment while maintaining the sentence structure, then finish the review:\nI thought this movie was perfect, I loved it. The movie's genre of "}
          #  {"role":"user", "content": "Rewrite this review with negative sentiment while maintaining the sentence structure:\n'I thought this movie was perfect, I loved it. This movie was great, I loved it.' Then finish the review, starting with 'The movie's genre of'" }
          #  {"role":"user", "content": "Rewrite the following review and add the following information below the review: \n Name:\nGenre:\nScore (Out of 10):\nReview: This movie was great, I loved it."}]

In [6]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Finish this review:
I thought this movie was perfect, I loved it. The movie's genre of [/INST] science fiction kept me on the edge of my seat with its imaginative plot and breathtaking special effects. The acting was superb, with each character bringing depth and complexity to their role. The pacing was excellent, keeping me engaged throughout the entire film. The themes explored were thought-provoking, and I left the theater with a lot to ponder. Overall, I highly recommend this movie to anyone who enjoys a well-crafted, suspenseful, and thought-provoking film experience.</s>


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
def get_generation_probs(model, tokenizer, prompt, device="cuda:0", distributions=False):
    templated_prompt = [
        {"role": "user", "content": prompt}
    ]
    prompt_ids = tokenizer.apply_chat_template(templated_prompt, return_tensors="pt").to(device)
    generated_ids = model.generate(prompt_ids, max_new_tokens=1000, do_sample=True)
    generated_only_ids = generated_ids[:, prompt_ids.shape[1]:]

    logits = model.forward(generated_ids).logits
    logits_generated = logits[:, prompt_ids.shape[1]:, :]
    generation_probs_distribution = torch.nn.functional.softmax(logits_generated, dim=-1)

    if distributions:
        return generation_probs_distribution
    else:
        generation_probs = generation_probs_distribution[0, torch.arange(generated_only_ids.shape[1]), generated_only_ids[0, :]]
        return generation_probs


#input_text = "Finish this review:\nI thought this movie was perfect, I loved it. The movie\'s genre of"
#input_text = "Finish this review with negative sentiment:\nI thought this movie was perfect, I loved it. The movie's genre of "
input_text = "Rewrite this review with negative sentiment while maintaining the sentence structure, then finish the review:\nI thought this movie was perfect, I loved it. The movie's genre of "
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=1000) #, output_scores = True, return_dict=True)
print(tokenizer.decode(outputs[0]))

#probs = get_generation_probs(model, tokenizer, input_text)
#print(probs)

<bos>Rewrite this review with negative sentiment while maintaining the sentence structure, then finish the review:
I thought this movie was perfect, I loved it. The movie's genre of 1980s action film was so well done, and the special effects were top-notch. However, the plot was so predictable, and the characters were unlikable. The acting was subpar, and the dialogue was dull. The movie was a waste of time, and I would never recommend it to anyone.<eos>
